In this notebook we would be merging two datasets MRDS and Minerals more spefically merging the country,state,county,and material coloum in both datasets and creating a new table called merged_geographical data.


In [ ]:
%%bigquery
select * from retails_stg.Mrds
order by mrds_id
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,Dep_ID,url,mrds_id,mas_id,site_name,latitude,longitude,region,country,state,county,com_type,commod1,commod2,commod3,data_source,load_time
0,10306836,https://mrdata.usgs.gov/mrds/show-mrds.php?dep...,None,None,Milton Silver Mine,44.52727,-68.22983,NA,United States,Maine,Hancock,M,"Copper, Silver, Lead",None,None,usgs,2024-02-02 22:02:24.030800+00:00
1,10306848,https://mrdata.usgs.gov/mrds/show-mrds.php?dep...,None,None,West Bay Mine,44.48659,-68.04614,NA,United States,Maine,Hancock,M,"Lead, Silver",None,None,usgs,2024-02-02 22:02:24.030800+00:00
2,10306849,https://mrdata.usgs.gov/mrds/show-mrds.php?dep...,None,None,Boss O' the Bay,44.47670,-68.16378,NA,United States,Maine,Hancock,M,"Lead, Copper",None,None,usgs,2024-02-02 22:02:24.030800+00:00
3,10306852,https://mrdata.usgs.gov/mrds/show-mrds.php?dep...,None,None,Manhattan Mine,44.36453,-68.74084,NA,United States,Maine,Hancock,M,"Gold, Silver, Copper",None,None,usgs,2024-02-02 22:02:24.030800+00:00
4,10307124,https://mrdata.usgs.gov/mrds/show-mrds.php?dep...,None,None,Johnson,34.71234,-81.90956,NA,United States,South Carolina,Spartanburg,N,Vermiculite,None,None,usgs,2024-02-02 22:02:24.030800+00:00


In [ ]:
%%bigquery
select * from retails_stg.Mineral
order by commod3
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,site_name,latitude,longitude,region,country,state,county,com_type,commod1,commod2,commod3,data_source,load_time
0,Unidentified Mine,-22.24588,-68.87284,SA,Chile,None,None,M,None,"Copper, Silver, Gold",None,kaggle,2024-02-02 22:02:37.360420+00:00
1,La Vega,5.70572,79.28114,SA,Peru,None,None,None,None,None,None,kaggle,2024-02-02 22:02:37.360420+00:00
2,Castle Mountain,NaN,NaN,None,None,None,None,None,None,None,None,kaggle,2024-02-02 22:02:37.360420+00:00
3,Darreh Hos Prospect,9.98400,44.91775,AF,Somalia,None,None,M,None,"Niobium (Columbium), Beryllium",None,kaggle,2024-02-02 22:02:37.360420+00:00
4,Paramo,5.71602,79.20724,SA,Peru,None,None,None,None,None,None,kaggle,2024-02-02 22:02:37.360420+00:00


In [ ]:
%%bigquery
select (select count(*) from retails_raw.mrds
        where country is not null) as mrds_country_count,
        (select count(*) from retails_stg.Mineral
        where country is not null) as mineral_country_count

Query is running:   0%|          |

Downloading:   0%|          |

,mrds_country_count,mineral_country_count
0,304617,304613


In [ ]:
%%bigquery
select (select count(*) from retails_raw.mrds
        where state is not null) as mrds_state_count,
        (select count(*) from retails_stg.Mineral) as mineral_state_count

Query is running:   0%|          |

Downloading:   0%|          |

,mrds_state_count,mineral_state_count
0,292148,304621


In [ ]:
%%bigquery
select (select count(*) from retails_stg.Mrds
        where commod1 is not null) as mrds_commod1_name_count,
        (select count(*) from retails_stg.Mineral
        where commod1 is not null) as mineral_commod1_name_count

Query is running:   0%|          |

Downloading:   0%|          |

,mrds_commod1_name_count,mineral_commod1_name_count
0,283601,283593


more commod1 in mrds than minerals

In [ ]:
%%bigquery
select (select count(*) from retails_stg.Mrds
        where commod2 is not null) as mrds_commod2_name_count,
        (select count(*) from retails_stg.Mineral
        where commod2 is not null) as mineral_commod2_name_count

Query is running:   0%|          |

Downloading:   0%|          |

,mrds_commod2_name_count,mineral_commod2_name_count
0,21632,21630


mrds.commod2 > mineral.commod2

Checking to see if site names count in mrds and mineral are the same.

In [ ]:
%%bigquery
select (select count(*) from retails_stg.Mineral
        where site_name is not null) as mineral_site_count,
        (select count(*) from retails_stg.Mrds
        where site_name is not null) as mrds_site_count


Query is running:   0%|          |

Downloading:   0%|          |

,mineral_site_count,mrds_site_count
0,304621,304621


In [ ]:
%%bigquery
select (select count(*) from retails_stg.Mineral
        where data_source is not null) as mineral_data_source,
        (select count(*) from retails_stg.Mrds
        where data_source is not null) as mrds_data_source

Query is running:   0%|          |

Downloading:   0%|          |

,mineral_data_source,mrds_data_source
0,304621,304632


##Creating Merged Table:

Merged country from minerals and mrds and matched commodoties, country and site name

In [ ]:
%%bigquery
  select mi.*, mr.country from retails_stg.Mineral mi
  join retails_raw.mrds mr
  on mi.site_name = mr.site_name and mi.country = mr.country and mi.commod1 = mr.commod1
  where mr.country is not null
  limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,site_name,latitude,longitude,region,country,state,county,com_type,commod1,commod2,commod3,data_source,load_time,country_1
0,Pompai Group Deposit,33.37615,-116.42024,NA,United States,California,San Diego,N,Ash,None,None,kaggle,2024-02-02 22:02:37.360420+00:00,United States
1,Western Platinum,-25.67996,27.53478,AF,South Africa,None,None,M,PGE,"Gold, Copper, Nickel",None,kaggle,2024-02-02 22:02:37.360420+00:00,South Africa
2,Brakpan Ref-Western Pt.,-26.16331,28.03475,AF,South Africa,None,None,M,PGE,Gold,None,kaggle,2024-02-02 22:02:37.360420+00:00,South Africa
3,Silet,22.66663,4.50156,AF,Algeria,None,None,M,PGE,None,None,kaggle,2024-02-02 22:02:37.360420+00:00,Algeria
4,Hoggar,22.99991,5.00157,AF,Algeria,None,None,M,PGE,None,Uranium,kaggle,2024-02-02 22:02:37.360420+00:00,Algeria
5,(Facility) Rustenburg Smelter,-25.68400,27.32700,AF,South Africa,None,None,M,PGE,"Nickel, Smelter, Gold, Copper",None,kaggle,2024-02-02 22:02:37.360420+00:00,South Africa
6,(Facility) Krasnoyarsk Pgm Refinery,56.01441,92.83330,EU,Russia,None,None,M,PGE,None,None,kaggle,2024-02-02 22:02:37.360420+00:00,Russia
7,Yenshan,36.56139,103.83393,AS,China,None,None,M,PGE,None,Copper,kaggle,2024-02-02 22:02:37.360420+00:00,China
8,Daigo,36.49863,138.99872,AS,Japan,None,None,M,PGE,None,"Iron, Gold",kaggle,2024-02-02 22:02:37.360420+00:00,Japan
9,Impala Platinum,-25.54664,27.21812,AF,South Africa,None,None,M,PGE,"Gold, Copper, Nickel",None,kaggle,2024-02-02 22:02:37.360420+00:00,South Africa


Logic on the creation of this table: Focuses on records from the United States, excluding territories like Puerto Rico.
Ensures commodities are specified and match between the datasets.
Includes only records with complete state and country information

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE retails_stg.merged_geographical_data AS
SELECT mi.*, mr.country AS mrds_country, #mr.data_source AS mrds_datasource,
FROM retails_stg.Mineral mi
left join retails_stg.Mrds mr
ON mi.site_name = mr.site_name
   AND mi.country = mr.country
   AND mi.commod1 = mr.commod1
   AND mi.data_source != mr.data_source;


Query is running:   0%|          |

""


In [ ]:
%%bigquery
SELECT
  (SELECT COUNT(*) FROM retails_stg.merged_geographical_data) AS total_datasource_count,
  (SELECT COUNT(DISTINCT data_source) FROM retails_stg.merged_geographical_data) AS unique_data_source_count;


Query is running:   0%|          |

Downloading:   0%|          |

,total_datasource_count,unique_data_source_count
0,45068210,1


## Redesigning Minerals


In [ ]:
%%bigquery
CREATE OR REPLACE TABLE retails_stg.Minerals AS
SELECT mi.*, mr.country AS mrds_country, #mr.data_source AS mrds_datasource,
FROM retails_stg.Mineral mi
left join retails_stg.Mrds mr
ON mi.site_name = mr.site_name
   AND mi.country = mr.country
   AND mi.commod1 = mr.commod1
   AND mi.data_source != mr.data_source;


# Foreign and Primary Key

In [ ]:
%%bigquery
alter table retails_stg.merged_geographical_data
  add primary key (site_name) not enforced

Query is running:   0%|          |

""


In [ ]:
%%bigquery
alter table retails_stg.Mrds add foreign key (state)
  references retails_stg.merged_geographical_data (site_name) not enforced;

Query is running:   0%|          |

""


## Clean Up

In [ ]:
%%bigquery
drop table retails_stg.Mrds

##Dropping Inventory table


In [ ]:
%%bigquery
drop table retails_stg.Inventory

##Dropping Minerals Table

In [ ]:

%%bigquery
drop table retails_stg.Minerals